In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import json
import datetime

In [2]:
Path = "C:/Users/schen414/Desktop/DataDive_2019/"

In [ ]:
%load_ext autotime

In [3]:
NYC_311_2018_2019_SEP_DF = pd.read_csv(Path+'data/NYC_311_20180101_through_20190930.csv')

C:\Users\schen414\AppData\Local\Continuum\Anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8,17,20,31,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
NYC_311_2018_2019_SEP_DF.sample(2)

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
4653444,43919434,09/28/2019 09:00:38 AM,09/28/2019 09:00:38 AM,DOB,Department of Buildings,Special Projects Inspection Team (SPIT),Illegal Hotel Rooms In Residential Building,NaN,10027,62 WEST 120 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.804128,-73.946751,"(40.80412798192669, -73.94675051045573)"
865452,39043984,04/27/2018 04:27:00 PM,04/27/2018 05:20:00 PM,DOT,Department of Transportation,Traffic Signal Condition,Controller,NaN,11354,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.764084,-73.833351,"(40.76408387781392, -73.83335145847988)"


In [5]:
NYC_311_2018_2019_SEP_DF.columns

Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'BBL', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Open Data Channel Type',
       'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
       'Latitude', 'Longitude', 'Location'],
      dtype='object')

In [6]:
NYC_311_2018_2019_SEP_DF['Descriptor_Length']=NYC_311_2018_2019_SEP_DF['Descriptor'].astype(str).apply(len)

In [7]:
NYC_311_2018_2019_SEP_DF['Descriptor_Length'].describe().apply(lambda x: format(x, 'f'))

count    4658607.000000
mean          19.154470
std           10.607182
min            3.000000
25%           12.000000
50%           16.000000
75%           26.000000
max           80.000000
Name: Descriptor_Length, dtype: object

In [8]:
NYC_311_2018_2019_SEP_DF['Complaint Type'].value_counts().reset_index(name="count").query("count > 1")

,index,count
0,Noise - Residential,390125
1,HEAT/HOT WATER,355410
2,Illegal Parking,309536
3,Request Large Bulky Item Collection,279854
4,Blocked Driveway,236969
5,Street Condition,175810
6,Noise - Street/Sidewalk,155109
7,UNSANITARY CONDITION,134652
8,Street Light Condition,128407
9,Water System,127552


In [9]:
NYC_311_2018_2019_SEP_DF[NYC_311_2018_2019_SEP_DF['Descriptor_Length']>75]['Descriptor'].sample(2)

3127995    Safety Netting/Guard Rails - Damaged/Inadequat...
3055564    Safety Netting/Guard Rails - Damaged/Inadequat...
Name: Descriptor, dtype: object

## Filtering down to the top 10 agencies in the data set for training/testing a MVP model (the rest of the agencies only take up 2.2% of the data set):

In [12]:
NYC_311_2018_2019_SEP_DF['Agency'].value_counts().reset_index(name="count").head(10)

,index,count
0,NYPD,1382354
1,HPD,958579
2,DSNY,657470
3,DOT,513786
4,DEP,346590
5,DOB,239063
6,DPR,203968
7,DOHMH,118165
8,DOF,78010
9,TLC,57537


In [14]:
TopTenAgency=NYC_311_2018_2019_SEP_DF['Agency'].value_counts().reset_index(name="count").head(10)['index'].tolist()

In [15]:
AgencyLabels=range(0,10)
AgencyLabels_List=[i for i in AgencyLabels]
AgencyLabels_List

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [16]:
TopTenAgency_DF=NYC_311_2018_2019_SEP_DF[NYC_311_2018_2019_SEP_DF.Agency.isin(TopTenAgency)]

In [17]:
TopTenAgency_DF.shape

(4555522, 42)

In [18]:
TopTenAgency_DF_X=TopTenAgency_DF['Descriptor']
TopTenAgency_DF_Y=TopTenAgency_DF['Agency']

## Train-Test Split (using only 20% (931.7K records) as training set for this 1 day DataDive, ideally would train on 80% or more):

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(TopTenAgency_DF_X, TopTenAgency_DF_Y, test_size=0.80, random_state=42)

In [21]:
trainDF=pd.concat([X_train, y_train], axis=1)

In [22]:
testDF = pd.concat([X_test, y_test], axis=1)

In [23]:
trainDF.shape

(911104, 2)

## Set up GloVe embeddings

In [24]:
GLOVE_DIR = r'C:/Users/schen414/Desktop/DataDive_2019/glove.6B/' 

In [25]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


## Preparing the text data

In [26]:
labels_index=dict(zip(TopTenAgency, AgencyLabels_List)) 

In [27]:
labels_index

{'NYPD': 0,
 'HPD': 1,
 'DSNY': 2,
 'DOT': 3,
 'DEP': 4,
 'DOB': 5,
 'DPR': 6,
 'DOHMH': 7,
 'DOF': 8,
 'TLC': 9}

In [28]:
trainDF['label_int'] = trainDF['Agency'] 

In [29]:
trainDF['label_int'].replace(labels_index, inplace=True)

In [30]:
trainDF[['label_int','Agency']]

,label_int,Agency
215934,2,DSNY
3349115,3,DOT
4637262,1,HPD
2049826,0,NYPD
1017923,6,DPR
2871804,2,DSNY
696131,0,NYPD
2901817,3,DOT
4350951,0,NYPD
3706760,4,DEP


In [64]:
trainDF.Agency.value_counts()

NYPD     276529
HPD      191819
DSNY     131432
DOT      102762
DEP       69274
DOB       47499
DPR       40795
DOHMH     23892
DOF       15632
TLC       11470
Name: Agency, dtype: int64

time: 399 ms


In [32]:
texts = trainDF['Descriptor'].astype(str).tolist()
labels = trainDF.label_int.tolist()

In [33]:
labels = trainDF.label_int.tolist()

## Set parameters: training & hyperparameters

In [34]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 100
VALIDATION_SPLIT = 0.3
EMBEDDING_DIM = 100

In [35]:
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = np_utils.to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Using TensorFlow backend.


Found 1314 unique tokens.
Shape of data tensor: (911104, 100)
Shape of label tensor: (911104, 10)


In [36]:
%load_ext autotime

## Prepare embedding layer

In [37]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

time: 5.98 ms


In [38]:
embedding_matrix.shape

(1315, 100)

time: 15 ms


In [39]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

time: 66.3 ms


## Training (only did 2 epoch due to long run time)

In [40]:
from keras.layers import Input, Flatten, Dense
from keras.layers.convolutional import Conv1D, MaxPooling1D
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32') # keras.engine.input_layer.Input()
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 3, activation='relu')(embedded_sequences)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(128, 3, activation='relu')(x)
x = MaxPooling1D()(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='sigmoid')(x)

model_0 = Model(sequence_input, preds)
model_0.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

epoch_data = model_0.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, batch_size=128)
model_training_timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 637773 samples, validate on 273331 samples
Epoch 1/2
637773/637773 [==============================] - 1423s 2ms/step - loss: 0.1122 - acc: 0.9518 - val_loss: 0.1085 - val_acc: 0.9530
Epoch 2/2
637773/637773 [==============================] - 1508s 2ms/step - loss: 0.1093 - acc: 0.9527 - val_loss: 0.1086 - val_acc: 0.9530


NameError: name 'datetime' is not defined

time: 49min 5s


In [41]:
import datetime
model_training_timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

time: 1.99 ms


In [42]:
model_0.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 100)          131500    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 128)           38528     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 32, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 30, 128)           49280     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 10, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 8, 128)            49280     
__________

In [43]:
epoch_data.history.keys()
model_hist_df = pd.DataFrame() 
model_hist_df['val_loss'] = epoch_data.history['val_loss']
model_hist_df['val_acc'] = epoch_data.history['val_acc']
model_hist_df['loss'] = epoch_data.history['loss']
model_hist_df['acc'] = epoch_data.history['acc']
model_hist_df

,val_loss,val_acc,loss,acc
0,0.108483,0.953027,0.112161,0.951828
1,0.108554,0.952958,0.109331,0.952725


time: 103 ms


## Apply model to a subset of the test set (600k) for quick evaluation

In [44]:
testDF.shape

(3644418, 2)

time: 3.99 ms


In [45]:
testDF_sub=testDF.sample(600000)

time: 1.28 s


In [46]:
sequences = tokenizer.texts_to_sequences(testDF_sub['Descriptor'].astype(str).tolist())
pre_model_seq = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', pre_model_seq.shape)

model_out = model_0.predict(pre_model_seq)
model_out_df = round(pd.DataFrame(model_out), 3)
model_out_df.columns = TopTenAgency
model_out_df['label'] = model_out_df.idxmax(axis=1)

testDF_sub['Model_label'] = model_out_df['label'].tolist()

Shape of data tensor: (600000, 100)
time: 8min 3s


In [66]:
testDF_sub.sample(10)

,Descriptor,Agency,Model_label
339275,DOOR,HPD,HPD
3702229,Building Permit - None,DOB,DOB
1505402,1RB Missed Recycling - M/G/Pl,DSNY,DSNY
1915038,Graffiti,DSNY,HPD
3511508,Noise: Construction Before/After Hours (NM1),DEP,DEP
1474446,Chemical Odor (HD1),DEP,DEP
4483538,Partial Access,NYPD,HPD
1083324,Loud Music/Party,NYPD,NYPD
3685226,For One Address,DPR,DPR
3797091,No Access,NYPD,HPD


time: 50.9 ms


In [50]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

time: 2 ms


In [51]:
print(classification_report(testDF_sub['Agency'],testDF_sub['Model_label']))
print(confusion_matrix(testDF_sub['Agency'],testDF_sub['Model_label']))

              precision    recall  f1-score   support

         DEP       1.00      0.94      0.97     45824
         DOB       1.00      0.96      0.98     31315
         DOF       1.00      0.84      0.91     10273
       DOHMH       1.00      0.39      0.56     15574
         DOT       0.97      0.39      0.55     67546
         DPR       1.00      0.73      0.84     26892
        DSNY       0.97      0.87      0.92     86489
         HPD       0.45      1.00      0.62    126463
        NYPD       0.99      0.56      0.71    181980
         TLC       1.00      0.37      0.54      7644

    accuracy                           0.73    600000
   macro avg       0.94      0.70      0.76    600000
weighted avg       0.87      0.73      0.74    600000

[[ 42942      1      0      0    930      0   1913     38      0      0]
 [     0  29922      0      6      1      0      0   1380      6      0]
 [     0      0   8594      0      0      0      0   1471    208      0]
 [    19      0      4

In [56]:
model_out_df[model_out_df['label'] =='HPD'].sample(10)

,NYPD,HPD,DSNY,DOT,DEP,DOB,DPR,DOHMH,DOF,TLC,label
337242,0.292,0.43,0.042,0.15,0.0,0.005,0.024,0.033,0.006,0.016,HPD
33167,0.292,0.43,0.042,0.15,0.0,0.005,0.024,0.033,0.006,0.016,HPD
356018,0.292,0.43,0.042,0.15,0.0,0.005,0.024,0.033,0.006,0.016,HPD
125030,0.000,1.00,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.000,HPD
515481,0.292,0.43,0.042,0.15,0.0,0.005,0.024,0.033,0.006,0.016,HPD
342166,0.292,0.43,0.042,0.15,0.0,0.005,0.024,0.033,0.006,0.016,HPD
147045,0.292,0.43,0.042,0.15,0.0,0.005,0.024,0.033,0.006,0.016,HPD
502704,0.292,0.43,0.042,0.15,0.0,0.005,0.024,0.033,0.006,0.016,HPD
255081,0.292,0.43,0.042,0.15,0.0,0.005,0.024,0.033,0.006,0.016,HPD
146681,0.292,0.43,0.042,0.15,0.0,0.005,0.024,0.033,0.006,0.016,HPD


time: 189 ms


In [63]:
testDF_sub[testDF_sub.Agency!=testDF_sub.Model_label]

,Descriptor,Agency,Model_label
3655004,Miscellaneous,DOF,HPD
3628882,Pothole,DOT,HPD
134630,Loud Television,NYPD,HPD
1637482,Food Spoiled,DOHMH,HPD
884144,Recycling Electronics,DSNY,HPD
1811715,Lamppost Damaged,DOT,HPD
3692395,Wear & Tear,DOT,HPD
2172672,Sidewalk Violation,DOT,HPD
1376835,Banging/Pounding,NYPD,HPD
1172475,Partial Access,NYPD,HPD


time: 308 ms


## Results show promise for using deep learning models for classifying complaints based on descriptions into the appropriate agencies. As more data are used for training, the performance could increase. It is also worth noting that the "descriptor" field seems like a pre-set selections of strings dependent on complaint types. In a case where an end-user can enter free text and 311 would like to generate/auto-populate/suggest other fields such as agencies/complaint types, a deep learning model can show even more advantages as the descriptions become less predictable and complex.